# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.2/884.2 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [2]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

con = duckdb.connect('nyc_data.db')

con.install_extension('spatial')
con.load_extension('spatial')

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 70.8MB/s]


Extracting files...


In [3]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [7]:
con.sql("SELECT * from nyc_subway_stations LIMIT 5")

┌──────────┬────────┬──────────────┬─────────────────┬─────────────────┬────────────────────────────────────────┬──────────────────────────────────┬───────────┬─────────┬─────────┬───────────┬─────────┬─────────┬─────────┬─────────────────────────────────────────────┐
│ OBJECTID │   ID   │     NAME     │    ALT_NAME     │    CROSS_ST     │               LONG_NAME                │              LABEL               │  BOROUGH  │ NGHBHD  │ ROUTES  │ TRANSFERS │  COLOR  │ EXPRESS │ CLOSED  │                    geom                     │
│  double  │ double │   varchar    │     varchar     │     varchar     │                varchar                 │             varchar              │  varchar  │ varchar │ varchar │  varchar  │ varchar │ varchar │ varchar │                  geometry                   │
├──────────┼────────┼──────────────┼─────────────────┼─────────────────┼────────────────────────────────────────┼──────────────────────────────────┼───────────┼─────────┼─────────┼───────────┼─

In [16]:
con.sql("SELECT * from nyc_neighborhoods WHERE name = 'Little Italy' ")

┌───────────┬──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │     NAME     │                                                                                                                                        geom                                                                                                                                         │
│  varchar  │   varchar    │                                                                                                                                      geometry                                                                                                                                       │
├───────────┼──────────────┼───────────────────────────────────────────────────

In [33]:
con.sql("""SELECT s.NAME, s.ROUTES FROM nyc_subway_stations AS s JOIN nyc_neighborhoods AS n ON ST_Contains(n.geom,s.geom) WHERE n.name = 'Little Italy'; """)

┌───────────┬─────────┐
│   NAME    │ ROUTES  │
│  varchar  │ varchar │
├───────────┼─────────┤
│ Spring St │ 6       │
└───────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [21]:
con.sql("""SELECT DISTINCT n.NAME FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE s.routes = '6' """)

┌───────────────────┐
│       NAME        │
│      varchar      │
├───────────────────┤
│ Chinatown         │
│ Greenwich Village │
│ Midtown           │
│ Upper East Side   │
│ Yorkville         │
│ Hunts Point       │
│ South Bronx       │
│ Soundview         │
│ Parkchester       │
│ Little Italy      │
│ Gramercy          │
│ East Harlem       │
│ Mott Haven        │
├───────────────────┤
│      13 rows      │
└───────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [23]:
con.sql("SELECT * from nyc_census_blocks LIMIT 5")

┌─────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│      BLKID      │ POPN_TOTAL │ POPN_WHITE │ POPN_BLACK │ POPN_NATIV │ POPN_ASIAN │ POPN_OTHER │   BORONAME    │               

In [25]:
con.sql("""SELECT sum(c.POPN_TOTAL) FROM nyc_census_blocks AS c JOIN nyc_neighborhoods AS s ON ST_Intersects(c.geom,s.geom) WHERE s.name = 'Battery Park' """)

┌───────────────────┐
│ sum(c.POPN_TOTAL) │
│      int128       │
├───────────────────┤
│             17153 │
└───────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [38]:
con.sql("""SELECT Sum(c.POPN_TOTAL)/ ST_Area(n.geom) AS popden , n.NAME
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods as n
ON ST_Intersects(c.geom,n.geom)
GROUP BY n.name, n.geom
ORDER BY popden DESC """)


┌───────────────────────┬─────────────────────────────┐
│        popden         │            NAME             │
│        double         │           varchar           │
├───────────────────────┼─────────────────────────────┤
│   0.06843513283772679 │ North Sutton Area           │
│   0.05040448341332536 │ East Village                │
│  0.048825180550629704 │ Chinatown                   │
│  0.048543725404414775 │ Carnegie Hill               │
│   0.04852448774898572 │ Upper East Side             │
│   0.04676934738855346 │ Little Italy                │
│   0.04283592043121245 │ Greenwich Village           │
│   0.04181238571997337 │ Gramercy                    │
│   0.04068792284068866 │ Morris Heights              │
│   0.04015248960800237 │ Upper West Side             │
│            ·          │      ·                      │
│            ·          │      ·                      │
│            ·          │      ·                      │
│  0.000924040052896441 │ Howland Hook          

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [39]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00


In [40]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [41]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [55]:
con.sql("CREATE TABLE ais AS SELECT vesselid, date, lat, lon FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv')")

In [56]:
con.sql("""ALTER TABLE ais ADD COLUMN geometry geometry""")



In [58]:
con.sql("CREATE INDEX ais_index ON ais USING RTREE(geometry)")

In [53]:
con.sql("DROP TABLE vinfo")
con.sql("CREATE TABLE vinfo AS SELECT vesselid, vessel_name, vsl_descr, v_length, draught, sog FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv')")

In [59]:
con.sql("SELECT * from ais LIMIT 5")

┌──────────┬─────────────────────┬──────────────────┬──────────────────┬──────────┐
│ vesselid │        date         │       lat        │       lon        │ geometry │
│  int64   │      timestamp      │      double      │      double      │ geometry │
├──────────┼─────────────────────┼──────────────────┼──────────────────┼──────────┤
│   350053 │ 2022-07-25 02:53:29 │ 45.1517766666667 │ 36.5133266666667 │ NULL     │
│   350053 │ 2022-07-25 03:09:37 │ 45.1464866666667 │         36.52078 │ NULL     │
│   350053 │ 2022-07-25 03:13:58 │ 45.1462183333333 │        36.521965 │ NULL     │
│   350053 │ 2022-07-25 04:16:06 │ 45.1450583333333 │         36.52202 │ NULL     │
│   350053 │ 2022-07-25 05:20:17 │ 45.1449333333333 │ 36.5218483333333 │ NULL     │
└──────────┴─────────────────────┴──────────────────┴──────────────────┴──────────┘

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.